## Download HoneyPot_dataSet

In [ ]:
!wget http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip

--2022-06-25 08:34:42--  http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip
Resolving infolab.tamu.edu (infolab.tamu.edu)... 45.55.217.29
Connecting to infolab.tamu.edu (infolab.tamu.edu)|45.55.217.29|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip [following]
--2022-06-25 08:34:43--  https://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip
Connecting to infolab.tamu.edu (infolab.tamu.edu)|45.55.217.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 263659672 (251M) [application/zip]
Saving to: ‘social_honeypot_icwsm_2011.zip’

social_honeypot_icw 100%[===================>] 251.45M  13.4MB/s    in 23s     

2022-06-25 08:35:08 (10.7 MB/s) - ‘social_honeypot_icwsm_2011.zip’ saved [263659672/263659672]



In [ ]:
!unzip social_honeypot_icwsm_2011.zip

Archive:  social_honeypot_icwsm_2011.zip
   creating: social_honeypot_icwsm_2011/
  inflating: social_honeypot_icwsm_2011/content_polluters.txt  
   creating: __MACOSX/
   creating: __MACOSX/social_honeypot_icwsm_2011/
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters.txt  
  inflating: social_honeypot_icwsm_2011/content_polluters_followings.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters_followings.txt  
  inflating: social_honeypot_icwsm_2011/content_polluters_tweets.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._content_polluters_tweets.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._legitimate_users.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users_followings.txt  
  inflating: __MACOSX/social_honeypot_icwsm_2011/._legitimate_users_followings.txt  
  inflating: social_honeypot_icwsm_2011/legitimate_users_tweets.txt  
  inflating: __MACOSX/soci

In [ ]:
!pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import re, string

from scipy.stats import zscore
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import scikitplot as skplt
import matplotlib.pyplot as plt

In [ ]:

def func(string):
   return pd.Series(string.split(',')).astype(int)
  
  
dfContent_polluters = pd.read_csv('social_honeypot_icwsm_2011/content_polluters.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"],index_col=0)

dfContent_polluters_followings = pd.read_csv('social_honeypot_icwsm_2011/content_polluters_followings.txt',sep='\t', names=
                 ["UserID","SeriesOfNumberOfFollowings"],converters={"SeriesOfNumberOfFollowings":func},index_col=0)

dfContent_polluters_tweets = pd.read_csv('social_honeypot_icwsm_2011/content_polluters_tweets.txt',sep='\t', names=
                 ["UserID","TweetID","Tweet","CreatedAt"], parse_dates=["CreatedAt"],index_col=0)

dfLegitimate_users = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users.txt',sep='\t', names=
                 ["UserID","CreatedAt","CollectedAt","NumberOfFollowings","NumberOfFollowers",
                  "NumberOfTweets","LengthOfScreenName","LengthOfDescriptionInUserProfile"],
                parse_dates=["CreatedAt","CollectedAt"],index_col=0)

dfLegitimate_users_followings = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users_followings.txt',sep='\t', names=
                 ["UserID","SeriesOfNumberOfFollowings"],converters={"SeriesOfNumberOfFollowings":func},index_col=0)


dfLegitimate_users_tweets = pd.read_csv('social_honeypot_icwsm_2011/legitimate_users_tweets.txt',sep='\t', names=
                 ["UserID","TweetID","Tweet","CreatedAt"], parse_dates=["CreatedAt"],index_col=0)

In [ ]:
def get_user_id(data_frame) :
  df = pd.DataFrame({"UserID":data_frame["UserID"]})
  return df

## 	Featurse Extraction


### 1 - The	longevity	of	the	account

In [ ]:

def longevity(data_frame) :
  df=pd.DataFrame({"LongevityOfAccount": (data_frame["CollectedAt"]-data_frame["CreatedAt"])/np.timedelta64(1, 'D')})
  return df

### 2 - Average Tweet Length

In [ ]:
def avgTweetLength(data_frame) :
  dftemp=pd.DataFrame({"AvgLengthTweets": data_frame["Tweet"].astype(str).apply(len)})
  df = dftemp.groupby(["UserID"]).mean()
  return df

### 3 -  |ReTweets |  /  |Tweets|

In [ ]:
def existURL(tweet) :
  if(len(re.findall('(RT|retweet|from|via)(?:\b\W*@(\w+))+', str(tweet)))==0):
    return 0;
  else:
    return 1;
  
def reTweetsFraction(data_frame) :
  dftemp=pd.DataFrame({"ReTweetFraction": data_frame["Tweet"].apply(existURL)})
  df = dftemp.groupby(["UserID"]).mean()
  return df
# def nbrOfTweets(data_frame) :
#   df = pd.DataFrame({"NumberOfTweets": data_frame["NumberOfTweets"] })
#   return df

# def nbrOfReTweets(data_frame) :
#   df = pd.DataFrame({"NumberOfReTweets": data_frame["NumberOfReTweets"] })
#   return df
  
# def reTweetsFraction(data_frame) :
#   dftemp=pd.DataFrame({"ReTweetFraction": (nbrOfReTweets(data_frame) / nbrOfTweets(data_frame)) })
#   df = dftemp.groupby(["UserID"]).mean()
#   return df


### 4 - |Tweets with URL|  /  |Tweets|

In [ ]:
def existURL(tweet) :
  if(len(re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', str(tweet)))==0):
    return 0;
  else:
    return 1;
  
def URLsPerTweets(data_frame) :
  dftemp=pd.DataFrame({"AvgTweetsWithURL": data_frame["Tweet"].apply(existURL)})
  df = dftemp.groupby(["UserID"]).mean()
  return df


### 5 - Average time between 2 consecutives tweets

In [ ]:
def avgTimeBetween2Tweets(data_frame):
  temp=pd.DataFrame({"CreatedAt":data_frame["CreatedAt"]}).sort_values(by=['UserID','CreatedAt'])
  dftemp=pd.DataFrame({"PreviousTweet":temp.groupby(['UserID']).apply(lambda x:x)['CreatedAt'],
                       "NextTweet":temp.groupby(['UserID']).shift(-1)['CreatedAt']})
  df=pd.DataFrame({"AvgMinutesBetween2Tweets": (dftemp["NextTweet"]-dftemp["PreviousTweet"]) / np.timedelta64(1, 'm')})
  df=df.groupby(["UserID"]).mean()
  return df

### 6 -  Followings / Followers

In [ ]:
def fractionFF(data_frame) :
  df = pd.DataFrame({"RatioFollowings/Followers": (data_frame["NumberOfFollowings"] / data_frame["NumberOfFollowers"]) })
  return df 

### 7 - The standard	deviation	of unique	numerical	IDs	of following

In [ ]:
def std(x):
  return x.std()

def followingsSTD(data_frame) :
  df = pd.DataFrame({"STDOfUniqueFollowers":data_frame["SeriesOfNumberOfFollowings"].apply(std)})
  return df

### 8 - Number of tweets / day

In [ ]:


def numberOfTweetsPerDay(data_frame) :
  dftemp = pd.DataFrame({"CreatedAt":data_frame["CreatedAt"].dt.date})
  query=dftemp.groupby(["UserID","CreatedAt"]).size()
  query2=query.groupby(["UserID"]).mean().reset_index(name='0')
  df = pd.DataFrame({"UserID":query2["UserID"],"NumberOfTweetsPerDay":query2["0"]})
  df=df.set_index("UserID")
  return df


### 9 - Average mentions / Tweet

In [ ]:
def countM(tweet):
  return len(re.findall('\s([@#][\w_-]+)', str(tweet)))

def mentionsPerTweets(data_frame):
  dftemp=pd.DataFrame({"AvgMentionsPerTweet": data_frame["Tweet"].apply(countM)})
  df = dftemp.groupby(["UserID"]).mean()
  return df

### 4 - Numbers of followings

In [ ]:
def nbrFollowings(data_frame) :
  df = pd.DataFrame({"NumberOfFollowings":data_frame["NumberOfFollowings"]})
  return df

### 5 - Numbers of followers

In [ ]:
def nbrFollowers(data_frame) :
  df = pd.DataFrame({"NumberOfFollowers":data_frame["NumberOfFollowers"]})
  return df

In [ ]:
# first_features_polluters=pd.concat([longevity(dfContent_polluters),
#                                     fractionFF(dfContent_polluters),
#                                     reTweetsFraction(dfContent_polluters_tweets),
#                                     avgTweetLength(dfContent_polluters_tweets),
#                                     URLsPerTweets(dfContent_polluters_tweets),
#                                     avgTimeBetween2Tweets(dfContent_polluters_tweets)],axis=1)

# first_features_legitimate=pd.concat([longevity(dfLegitimate_users),
#                                     fractionFF(dfLegitimate_users),
#                                     reTweetsFraction(dfLegitimate_users_tweets),
#                                     avgTweetLength(dfLegitimate_users_tweets),
#                                     URLsPerTweets(dfLegitimate_users_tweets),
#                                     avgTimeBetween2Tweets(dfLegitimate_users_tweets)],axis=1)

first_features_polluters=pd.concat([longevity(dfContent_polluters),
                                    # fractionFF(dfContent_polluters),
                                    nbrFollowings(dfContent_polluters), #nbrFollowers(dfContent_polluters),
                                    followingsSTD(dfContent_polluters_followings),
                                    mentionsPerTweets(dfContent_polluters_tweets),
                                    reTweetsFraction(dfContent_polluters_tweets),
                                    avgTweetLength(dfContent_polluters_tweets),
                                    URLsPerTweets(dfContent_polluters_tweets),
                                    numberOfTweetsPerDay(dfContent_polluters_tweets),
                                    avgTimeBetween2Tweets(dfContent_polluters_tweets)],axis=1)

first_features_legitimate=pd.concat([longevity(dfLegitimate_users),
                                    # fractionFF(dfLegitimate_users),
                                    nbrFollowings(dfLegitimate_users),#nbrFollowers(dfLegitimate_users),
                                    followingsSTD(dfLegitimate_users_followings),
                                    mentionsPerTweets(dfLegitimate_users_tweets),
                                    reTweetsFraction(dfLegitimate_users_tweets),
                                    avgTweetLength(dfLegitimate_users_tweets),
                                    URLsPerTweets(dfLegitimate_users_tweets),
                                    numberOfTweetsPerDay(dfLegitimate_users_tweets),
                                    avgTimeBetween2Tweets(dfLegitimate_users_tweets)],axis=1)

In [ ]:
first_features_polluters_wtRT=pd.concat([longevity(dfContent_polluters),
                                    # followingsSTD(dfContent_polluters_followings),
                                    mentionsPerTweets(dfContent_polluters_tweets),
                                    avgTweetLength(dfContent_polluters_tweets),
                                    URLsPerTweets(dfContent_polluters_tweets),
                                    numberOfTweetsPerDay(dfContent_polluters_tweets),
                                    avgTimeBetween2Tweets(dfContent_polluters_tweets)],axis=1)

first_features_legitimate_wtRT=pd.concat([longevity(dfLegitimate_users),
                                    # followingsSTD(dfLegitimate_users_followings),
                                    mentionsPerTweets(dfLegitimate_users_tweets),
                                    avgTweetLength(dfLegitimate_users_tweets),
                                    URLsPerTweets(dfLegitimate_users_tweets),
                                    numberOfTweetsPerDay(dfLegitimate_users_tweets),
                                    avgTimeBetween2Tweets(dfLegitimate_users_tweets)],axis=1)

## Label Honey Pot dataSet

In [ ]:
data_set_polluters = first_features_polluters
data_set_legitimate = first_features_legitimate

In [ ]:
data_set_polluters_wtRT = first_features_polluters_wtRT
data_set_legitimate_wtRT = first_features_legitimate_wtRT

In [ ]:
data_set_polluters.insert(0,"Class",1)
data_set_legitimate.insert(0,"Class",0)

In [ ]:
data_set_polluters_wtRT.insert(0,"Class",1)
data_set_legitimate_wtRT.insert(0,"Class",0)

## Clean Data

In [ ]:
def attributesOfUniqueValues(data_set_users) :
  list=[]
  for col in data_set_users.columns:
    if (data_set_users[col]==data_set_users[col].iloc[0]).all == True:
      list.append(col)
  return data_set_users.drop(list)

In [ ]:
def duplicateRows(data_set_users):
  return data_set_users.drop_duplicates()

In [ ]:
def missingValues(data_set_users):
  valuesReplaced = data_set_users.replace([np.inf,-np.inf],np.nan)
  valuesReplaced.fillna(0,inplace=True)
  return valuesReplaced

In [ ]:
def shiftCol(data_set_users):
  df=data_set_users.drop(["Class"],axis=1)
  # df=df.apply(zscore)
  df["Class"]=data_set_users["Class"]
  return df

In [ ]:
data_set_polluters=attributesOfUniqueValues(data_set_polluters)
data_set_polluters=duplicateRows(data_set_polluters)
data_set_polluters=missingValues(data_set_polluters)
data_set_polluters=shiftCol(data_set_polluters)

In [ ]:
data_set_legitimate=attributesOfUniqueValues(data_set_legitimate)
data_set_legitimate=duplicateRows(data_set_legitimate)
data_set_legitimate=missingValues(data_set_legitimate)
data_set_legitimate=shiftCol(data_set_legitimate)

In [ ]:
data_set=pd.concat([data_set_polluters,data_set_legitimate])

In [ ]:
data_set_polluters_wtRT=attributesOfUniqueValues(data_set_polluters_wtRT)
data_set_polluters_wtRT=duplicateRows(data_set_polluters_wtRT)
data_set_polluters_wtRT=missingValues(data_set_polluters_wtRT)
data_set_polluters_wtRT=shiftCol(data_set_polluters_wtRT)

data_set_legitimate_wtRT=attributesOfUniqueValues(data_set_legitimate_wtRT)
data_set_legitimate_wtRT=duplicateRows(data_set_legitimate_wtRT)
data_set_legitimate_wtRT=missingValues(data_set_legitimate_wtRT)
data_set_legitimate_wtRT=shiftCol(data_set_legitimate_wtRT)

data_set_wtRT=pd.concat([data_set_polluters_wtRT,data_set_legitimate_wtRT])

data_set_wtRT

,LongevityOfAccount,AvgMentionsPerTweet,AvgLengthTweets,AvgTweetsWithURL,NumberOfTweetsPerDay,AvgMinutesBetween2Tweets,Class
UserID,,,,,,,
6301,1217.812905,0.140000,90.555000,0.630000,4.000000,492.219765,1
10836,1329.539931,0.464646,84.686869,0.601010,2.224719,4396.333756,1
10997,1272.640301,0.791878,98.974619,0.263959,49.250000,28.432738,1
633293,1105.971852,0.290000,74.875000,0.320000,10.526316,125.861139,1
717883,1105.341435,0.080000,99.910000,0.680000,9.523810,218.854020,1
...,...,...,...,...,...,...,...
93390990,0.053056,0.800000,58.400000,0.800000,5.000000,15.833333,0
93402679,0.005995,0.000000,10.000000,0.000000,1.000000,0.000000,0
93419256,0.005069,0.000000,23.000000,0.000000,1.000000,0.000000,0


In [ ]:
data_set

,LongevityOfAccount,NumberOfFollowings,STDOfUniqueFollowers,AvgMentionsPerTweet,ReTweetFraction,AvgLengthTweets,AvgTweetsWithURL,NumberOfTweetsPerDay,AvgMinutesBetween2Tweets,Class
UserID,,,,,,,,,,
6301,1217.812905,3269,189.834956,0.140000,0.630000,90.555000,0.630000,4.000000,492.219765,1
10836,1329.539931,1949,11.384581,0.464646,0.601010,84.686869,0.601010,2.224719,4396.333756,1
10997,1272.640301,1119,859.067567,0.791878,0.263959,98.974619,0.263959,49.250000,28.432738,1
633293,1105.971852,2174,284.877349,0.290000,0.320000,74.875000,0.320000,10.526316,125.861139,1
717883,1105.341435,7731,114.229063,0.080000,0.680000,99.910000,0.680000,9.523810,218.854020,1
...,...,...,...,...,...,...,...,...,...,...
93390990,0.053056,5,0.501015,0.800000,0.800000,58.400000,0.800000,5.000000,15.833333,0
93402679,0.005995,20,0.000000,0.000000,0.000000,10.000000,0.000000,1.000000,0.000000,0
93419256,0.005069,0,0.000000,0.000000,0.000000,23.000000,0.000000,1.000000,0.000000,0


In [ ]:
X=data_set.drop(['Class'],axis=1)
X=X.values
Y=data_set["Class"].values
train, test, target, trueResult = train_test_split(X, Y, test_size=0.3, random_state=1)

In [ ]:
X_wtRT=data_set_wtRT.drop(['Class'],axis=1)
X_wtRT=X_wtRT.values
Y_wtRT=data_set_wtRT["Class"].values
train_wtRT, test_wtRT, target_wtRT, trueResult_wtRT = train_test_split(X_wtRT, Y_wtRT, test_size=0.3, random_state=1)

In [ ]:
# shuffle = np.random.permutation(60000)
# train, target = train[shuffle], target[shuffle]

# train_wtRT, target_wtRT = train_wtRT[shuffle], target_wtRT[shuffle]

In [ ]:
train

array([[7.67862269e+00, 5.48000000e+02, 3.63368211e+02, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [3.53187488e+02, 2.14000000e+02, 2.08869679e+00, ...,
        2.55000000e-01, 1.33333333e+01, 1.12214489e+02],
       [3.51044236e+02, 4.68300000e+03, 2.35307987e+02, ...,
        2.95000000e-01, 2.70270270e+00, 7.30171692e+02],
       ...,
       [3.12887222e+02, 1.88100000e+03, 3.06642257e+01, ...,
        1.10000000e-01, 1.66666667e+01, 8.40231156e+01],
       [8.59500347e+01, 3.65000000e+02, 2.31430122e-01, ...,
        8.88888889e-01, 4.15384615e+00, 2.17179277e+03],
       [1.55452350e+02, 1.80000000e+02, 5.00919981e-01, ...,
        8.75000000e-01, 2.66666667e+00, 2.00476262e+04]])

In [ ]:
target

array([1, 0, 1, ..., 1, 1, 0])

**Application des Algorithmes**

In [ ]:

# # Decision Tree
# decisionTree = DecisionTreeClassifier(random_state=0)
# decisionTree.fit(train, target)
# predictedAD = decisionTree.predict(test)
# probasAD = decisionTree.predict_proba(test)

# # Random Forest
# randomForest = RandomForestClassifier(n_estimators=100)
# randomForest.fit(train, target)
# predictedRF = randomForest.predict(test)
# probasRF = randomForest.predict_proba(test)

# # Naive Bayes
# gnb = GaussianNB()
# gnb.fit(train, target)
# predictedNB = gnb.predict(test)
# probasNB = gnb.predict_proba(test)

# # # SVM linear
# # svm = SVC(kernel='linear',probability=True)
# # svm.fit(train, target)
# # predictedSVM = svm.predict(test)
# # probasSVM = svm.predict_proba(test)

# # # SVM sigmoid
# # svmS = SVC(kernel='sigmoid',probability=True)
# # svmS.fit(train, target)
# # predictedSVMS = svmS.predict(test)
# # probasSVMS = svmS.predict_proba(test)


In [ ]:

# from sklearn.neighbors import KNeighborsClassifier

# # KNN  
# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(train, target)
# predictedknn = knn.predict(test)
# probasknn = knn.predict_proba(test)

# # KNN  _wtRT
# knn_wtRT = KNeighborsClassifier(n_neighbors=5)
# knn_wtRT.fit(train_wtRT, target_wtRT)
# predictedknn_wtRT = knn_wtRT.predict(test_wtRT)
# probasknn_wtRT = knn_wtRT.predict_proba(test_wtRT)

In [ ]:
# # Decision Tree _wtRT
# decisionTree_wtRT = DecisionTreeClassifier(random_state=0)
# decisionTree_wtRT.fit(train_wtRT, target_wtRT)
# predictedAD_wtRT = decisionTree_wtRT.predict(test_wtRT)
# probasAD_wtRT = decisionTree_wtRT.predict_proba(test_wtRT)

# # Random Forest _wtRT
# randomForest_wtRT = RandomForestClassifier(n_estimators=100)
# randomForest_wtRT.fit(train_wtRT, target_wtRT)
# predictedRF_wtRT = randomForest_wtRT.predict(test_wtRT)
# probasRF_wtRT = randomForest_wtRT.predict_proba(test_wtRT)

# # Naive Bayes _wtRT
# gnb_wtRT = GaussianNB()
# gnb_wtRT.fit(train_wtRT, target_wtRT)
# predictedNB_wtRT = gnb_wtRT.predict(test_wtRT)
# probasNB_wtRT = gnb_wtRT.predict_proba(test_wtRT)

# # # SVM linear _wtRT
# # svm_wtRT = SVC(kernel='linear',probability=True)
# # svm_wtRT.fit(train_wtRT, target_wtRT)
# # predictedSVM_wtRT = svm.predict(test_wtRT)
# # probasSVM_wtRT = svm.predict_proba(test_wtRT)

# # # SVM sigmoid _wtRT
# # svmS_wtRT = SVC(kernel='sigmoid',probability=True)
# # svmS_wtRT.fit(train_wtRT, target_wtRT)
# # predictedSVMS_wtRT = svmS_wtRT.predict(test_wtRT)
# # probasSVMS_wtRT = svmS_wtRT.predict_proba(test_wtRT)

In [ ]:
# # SVM linear
# svm = SVC()
# svm.fit(train, target)
# predictedSVM = svm.predict(test)
# # probasSVM = svm.predict_proba(test)

# # SVM sigmoid
# svmS = SVC(kernel='sigmoid')
# svmS.fit(train, target)
# predictedSVMS = svmS.predict(test)
# # probasSVMS = svmS.predict_proba(test)

# # SVM linear _wtRT
# svm_wtRT = SVC()
# svm_wtRT.fit(train_wtRT, target_wtRT)
# predictedSVM_wtRT = svm_wtRT.predict(test_wtRT)
# # probasSVM_wtRT = svm_wtRT.predict_proba(test_wtRT)

# # SVM sigmoid _wtRT
# svmS_wtRT = SVC(kernel='sigmoid')
# svmS_wtRT.fit(train_wtRT, target_wtRT)
# predictedSVMS_wtRT = svmS_wtRT.predict(test_wtRT)
# # probasSVMS_wtRT = svmS_wtRT.predict_proba(test_wtRT)

In [ ]:
# svmL = SVC(kernel='linear')
# svmL.fit(train, target)
# predictedSVML = svmL.predict(test)

# # SVM linear _wtRT
# svmL_wtRT = SVC(kernel='linear')
# svmL_wtRT.fit(train_wtRT, target_wtRT)
# predictedSVML_wtRT = svmL_wtRT.predict(test_wtRT)

### Performances



> KNN


In [ ]:
# precisionknn = precision_score(trueResult, predictedknn)
# recallknn = recall_score(trueResult, predictedknn)
# # fmeasureknn  = f1_score(trueResult, predictedknn)
# fprknn , tprknn , thresholdsknn = roc_curve(trueResult, predictedknn)
# rocknn  = metrics.auc(fprknn , tprknn )
# print("Precision: %0.4f" % precisionknn)
# print("Recall: %0.4f" % recallknn)
# # print("F-mesure: %0.4f" % fmeasureknn)
# print("Surface sous la courbe ROC: %0.4f" % rocknn)
# skplt.metrics.plot_confusion_matrix(trueResult, predictedknn, normalize=True)
# skplt.metrics.plot_roc(trueResult, probasknn)
# skplt.metrics.plot_precision_recall(trueResult, probasknn)


In [ ]:
# precisionknn_wtRT = precision_score(trueResult_wtRT, predictedknn_wtRT)
# recallknn_wtRT = recall_score(trueResult_wtRT, predictedknn_wtRT)
# # fmeasureknn_wtRT  = f1_score(trueResult_wtRT, predictedknn_wtRT)
# fprknn_wtRT , tprknn_wtRT , thresholdsknn_wtRT = roc_curve(trueResult_wtRT, predictedknn_wtRT)
# rocknn_wtRT  = metrics.auc(fprknn_wtRT , tprknn_wtRT )
# print("Precision: %0.4f" % precisionknn_wtRT)
# print("Recall: %0.4f" % recallknn_wtRT)
# # print("F-mesure: %0.4f" % fmeasureknn_wtRT)
# print("Surface sous la courbe ROC: %0.4f" % rocknn_wtRT)
# skplt.metrics.plot_confusion_matrix(trueResult_wtRT, predictedknn_wtRT, normalize=True)
# # skplt.metrics.plot_roc(trueResult_wtRT, probasknn_wtRT)
# # skplt.metrics.plot_precision_recall(trueResult_wtRT, probasKnn_wtRT)



> Arbre de décision



In [ ]:
# precisionAD = precision_score(trueResult, predictedAD)
# recallAD = recall_score(trueResult, predictedAD)
# fmeasureAD = f1_score(trueResult, predictedAD)
# fprAD, tprAD, thresholdsAD = roc_curve(trueResult, predictedAD)
# rocAD = metrics.auc(fprAD, tprAD)
# print("Precision: %0.4f" % precisionAD)
# print("Recall: %0.4f" % recallAD)
# print("F-mesure: %0.4f" % fmeasureAD)
# print("Surface sous la courbe ROC: %0.4f" % rocAD)
# skplt.metrics.plot_confusion_matrix(trueResult, predictedAD, normalize=True)
# skplt.metrics.plot_roc(trueResult, probasAD)
# skplt.metrics.plot_precision_recall(trueResult, probasAD)




> Arbre de décision _wtRT 



In [ ]:
# precisionAD_wtRT  = precision_score(trueResult_wtRT , predictedAD_wtRT )
# recallAD_wtRT  = recall_score(trueResult_wtRT , predictedAD_wtRT )
# fmeasureAD_wtRT  = f1_score(trueResult_wtRT , predictedAD_wtRT )
# fprAD_wtRT , tprAD_wtRT , thresholdsAD_wtRT  = roc_curve(trueResult_wtRT , predictedAD_wtRT )
# rocAD_wtRT = metrics.auc(fprAD_wtRT , tprAD_wtRT )
# print("Precision_wtRT : %0.4f" % precisionAD_wtRT )
# print("Recall_wtRT : %0.4f" % recallAD_wtRT )
# print("F-mesure_wtRT : %0.4f" % fmeasureAD_wtRT )
# print("Surface sous la courbe ROC_wtRT : %0.4f" % rocAD_wtRT )
# skplt.metrics.plot_confusion_matrix(trueResult_wtRT , predictedAD_wtRT , normalize=True)
# skplt.metrics.plot_roc(trueResult_wtRT , probasAD_wtRT )
# skplt.metrics.plot_precision_recall(trueResult_wtRT , probasAD_wtRT )




> Random Forest



In [ ]:
# precisionRF = precision_score(trueResult, predictedRF)
# recallRF = recall_score(trueResult, predictedRF)
# fmeasureRF = f1_score(trueResult, predictedRF)
# fprRF, tprRF, thresholdsRF = roc_curve(trueResult, predictedRF)
# rocRF = metrics.auc(fprRF, tprRF)
# print("Precision: %0.4f" % precisionRF)
# print("Recall: %0.4f" % recallRF)
# print("F-mesure: %0.4f" % fmeasureRF)
# print("Surface sous la courbe ROC: %0.4f" % rocRF)
# skplt.metrics.plot_confusion_matrix(trueResult, predictedRF, normalize=True)
# skplt.metrics.plot_roc(trueResult, probasRF)
# skplt.metrics.plot_precision_recall(trueResult, probasRF)


> Random Forest _wtrt_wtRT




In [ ]:
# precisionRF_wtRT = precision_score(trueResult_wtRT, predictedRF_wtRT)
# recallRF_wtRT = recall_score(trueResult_wtRT, predictedRF_wtRT)
# fmeasureRF_wtRT = f1_score(trueResult_wtRT, predictedRF_wtRT)
# fprRF_wtRT, tprRF_wtRT, thresholdsRF_wtRT = roc_curve(trueResult_wtRT, predictedRF_wtRT)
# rocRF_wtRT = metrics.auc(fprRF_wtRT, tprRF_wtRT)
# print("Precision_wtRT: %0.4f" % precisionRF_wtRT)
# print("Recall_wtRT: %0.4f" % recallRF_wtRT)
# print("F-mesure_wtRT: %0.4f" % fmeasureRF_wtRT)
# print("Surface sous la courbe ROC_wtRT: %0.4f" % rocRF_wtRT)
# skplt.metrics.plot_confusion_matrix(trueResult_wtRT, predictedRF_wtRT, normalize=True)
# skplt.metrics.plot_roc(trueResult_wtRT, probasRF_wtRT)
# skplt.metrics.plot_precision_recall(trueResult_wtRT, probasRF_wtRT)



>Classification bayésienne naïve



In [ ]:
# precisionNB = precision_score(trueResult, predictedNB)
# recallNB = recall_score(trueResult, predictedNB)
# fmeasureNB = f1_score(trueResult, predictedNB)
# fprNB, tprNB, thresholdsNB = roc_curve(trueResult, predictedNB)
# rocNB = metrics.roc_auc_score(trueResult, predictedNB)
# print("Precision: %0.4f" % precisionNB)
# print("Recall: %0.4f" % recallNB)
# print("F-mesure: %0.4f" % fmeasureNB)
# print("Surface sous la courbe ROC: %0.4f" % rocNB)
# skplt.metrics.plot_confusion_matrix(trueResult, predictedNB, normalize=True)
# skplt.metrics.plot_roc(trueResult, probasNB)
# skplt.metrics.plot_precision_recall(trueResult, probasNB)



>Classification bayésienne naïve _wtRT



In [ ]:
# precisionNB_wtRT = precision_score(trueResult_wtRT, predictedNB_wtRT)
# recallNB_wtRT = recall_score(trueResult_wtRT, predictedNB_wtRT)
# fmeasureNB_wtRT = f1_score(trueResult_wtRT, predictedNB_wtRT)
# fprNB_wtRT, tprNB_wtRT, thresholdsNB_wtRT = roc_curve(trueResult_wtRT, predictedNB_wtRT)
# rocNB_wtRT = metrics.roc_auc_score(trueResult_wtRT, predictedNB_wtRT)
# print("Precision_wtRT: %0.4f" % precisionNB_wtRT)
# print("Recall_wtRT: %0.4f" % recallNB_wtRT)
# print("F-mesure_wtRT: %0.4f" % fmeasureNB_wtRT)
# print("Surface sous la courbe ROC: %0.4f" % rocNB_wtRT)
# skplt.metrics.plot_confusion_matrix(trueResult_wtRT, predictedNB_wtRT, normalize=True)
# skplt.metrics.plot_roc(trueResult_wtRT, probasNB_wtRT)
# skplt.metrics.plot_precision_recall(trueResult_wtRT, probasNB_wtRT)

In [ ]:
# print(trueResult.shape)

SVM


In [ ]:
# precisionSVM = precision_score(trueResult, predictedSVM)
# recallSVM = recall_score(trueResult, predictedSVM)
# fmeasureSVM = f1_score(trueResult, predictedSVM)
# fprSVM, tprSVM, thresholdsSVM = roc_curve(trueResult, predictedSVM)
# rocSVM = metrics.auc(fprSVM, tprSVM)
# print("Precision: %0.4f" % precisionSVM)
# print("Recall: %0.4f" % recallSVM)
# print("F-mesure: %0.4f" % fmeasureSVM)
# print("Surface sous la courbe ROC: %0.4f" % rocSVM)
# skplt.metrics.plot_confusion_matrix(trueResult, predictedSVM, normalize=True)

In [ ]:
# precisionSVM_wtRT = precision_score(trueResult_wtRT, predictedSVM_wtRT)
# recallSVM_wtRT = recall_score(trueResult_wtRT, predictedSVM_wtRT)
# fmeasureSVM_wtRT = f1_score(trueResult_wtRT, predictedSVM_wtRT)
# fprSVM_wtRT, tprSVM_wtRT, thresholdsSVM_wtRT = roc_curve(trueResult_wtRT, predictedSVM_wtRT)
# rocSVM_wtRT = metrics.auc(fprSVM_wtRT, tprSVM_wtRT)
# print("Precision_wtRT: %0.4f" % precisionSVM_wtRT)
# print("Recall_wtRT: %0.4f" % recallSVM_wtRT)
# print("F-mesure_wtRT: %0.4f" % fmeasureSVM_wtRT)
# print("Surface sous la courbe ROC_wtRT: %0.4f" % rocSVM_wtRT)
# skplt.metrics.plot_confusion_matrix(trueResult_wtRT, predictedSVM_wtRT, normalize=True)
# # skplt.metrics.plot_roc(trueResult, probasSVM)
# # skplt.metrics.plot_precision_recall(trueResult, probasSVM)

In [ ]:
# precisionSVML = precision_score(trueResult, predictedSVML)
# recallSVML = recall_score(trueResult, predictedSVML)
# fmeasureSVML = f1_score(trueResult, predictedSVML)
# fprSVML, tprSVML, thresholdsSVML = roc_curve(trueResult, predictedSVML)
# rocSVML = metrics.auc(fprSVML, tprSVML)
# print("Precision: %0.4f" % precisionSVML)
# print("Recall: %0.4f" % recallSVML)
# print("F-mesure: %0.4f" % fmeasureSVML)
# print("Surface sous la courbe ROC: %0.4f" % rocSVML)
# skplt.metrics.plot_confusion_matrix(trueResult, predictedSVML, normalize=True)

In [ ]:
# precisionSVML_wtRT = precision_score(trueResult_wtRT, predictedSVML_wtRT)
# recallSVML_wtRT = recall_score(trueResult_wtRT, predictedSVML_wtRT)
# fmeasureSVML_wtRT = f1_score(trueResult_wtRT, predictedSVML_wtRT)
# # fprSVM_wtRT, tprSVM_wtRT, thresholdsSVM_wtRT = roc_curve(trueResult_wtRT, predictedSVML_wtRT)
# # rocSVM_wtRT = metrics.auc(fprSVM_wtRT, tprSVM_wtRT)
# print("Precision_wtRT: %0.4f" % precisionSVML_wtRT)
# print("Recall_wtRT: %0.4f" % recallSVML_wtRT)
# print("F-mesure_wtRT: %0.4f" % fmeasureSVML_wtRT)
# # print("Surface sous la courbe ROC_wtRT: %0.4f" % rocSVM_wtRT)
# skplt.metrics.plot_confusion_matrix(trueResult_wtRT, predictedSVML_wtRT, normalize=True)

# SVM sigmoid

In [ ]:
# precisionSVM = precision_score(trueResult, predictedSVMS)
# recallSVM = recall_score(trueResult, predictedSVMS)
# fmeasureSVM = f1_score(trueResult, predictedSVMS)
# fprSVM, tprSVM, thresholdsSVM = roc_curve(trueResult, predictedSVMS)
# rocSVM = metrics.auc(fprSVM, tprSVM)
# print("Precision: %0.4f" % precisionSVM)
# print("Recall: %0.4f" % recallSVM)
# print("F-mesure: %0.4f" % fmeasureSVM)
# print("Surface sous la courbe ROC: %0.4f" % rocSVM)
# skplt.metrics.plot_confusion_matrix(trueResult, predictedSVMS, normalize=True)

In [ ]:
# precisionSVM_wtRT = precision_score(trueResult_wtRT, predictedSVMS_wtRT)
# recallSVM_wtRT = recall_score(trueResult_wtRT, predictedSVMS_wtRT)
# fmeasureSVM_wtRT = f1_score(trueResult_wtRT, predictedSVMS_wtRT)
# fprSVM_wtRT, tprSVM_wtRT, thresholdsSVM_wtRT = roc_curve(trueResult_wtRT, predictedSVMS_wtRT)
# rocSVM = metrics.auc(fprSVM, tprSVM)
# print("Precision_wtRT: %0.4f" % precisionSVM_wtRT)
# print("Recall_wtRT: %0.4f" % recallSVM_wtRT)
# print("F-mesure_wtRT: %0.4f" % fmeasureSVM_wtRT)
# print("Surface sous la courbe ROC: %0.4f" % rocSVM_wtRT)
# skplt.metrics.plot_confusion_matrix(trueResult_wtRT, predictedSVMS_wtRT, normalize=True)
# # skplt.metrics.plot_roc(trueResult, probasSVM)
# # skplt.metrics.plot_precision_recall(trueResult, probasSVMS)

In [ ]:
from sklearn import model_selection
from sklearn.metrics import classification_report,confusion_matrix,make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


In [ ]:


seed = 6
# scoring ='accuracy'

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

In [ ]:
# # With Out Rewteet faeture
# models = []
# models.append(('LR',LogisticRegression()))             #Logistic Regression
# models.append(('MLP',MLPClassifier(hidden_layer_sizes=(15,15))))             #Logistic Regression
# models.append(('LDA',LinearDiscriminantAnalysis()))    #Linear Discrimant Analysis
# models.append(('KNN|IBK',KNeighborsClassifier(n_neighbors=5)))          #K-NearestNeighbor Classifier
# models.append(('CART|J48',DecisionTreeClassifier()))       #Decision Tree Classifier 
# models.append(('RF',RandomForestClassifier(n_estimators=100)))       #Random Forest Classifier 
# models.append(('NB',GaussianNB()))                     #Naive Bayes
# models.append(('SVM',SVC()))                           #Support Vector Machine
# results = []
# names = []
# for name,model in models:
#     kfold = model_selection.KFold(n_splits=10,shuffle= False)
#     cv_results = model_selection.cross_validate(model,train_wtRT ,target_wtRT,cv=kfold, scoring= scoring)
#     results.append(cv_results)
#     names.append(name)
#     precision = '%s precision: %f (%f)' % (name, cv_results['test_precision'].mean()*100, cv_results['test_precision'].std())
#     recall = '%s recall: %f (%f)' % (name, cv_results['test_recall'].mean()*100, cv_results['test_recall'].std())
#     f1_score = '%s f1_score: %f (%f)' % (name, cv_results['test_f1_score'].mean()*100, cv_results['test_f1_score'].std())
#     print(precision)
#     print(recall)
#     print(f1_score)
# # dict_keys(['fit_time', 'score_time', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1_score'])
# # cross_validate or cross_val_score

In [ ]:

#USING CROSS VALIDATION TO ESTIMATE THE BEST ALGOTRITHM 
models = []
models.append(('LR',LogisticRegression()))             #Logistic Regression
models.append(('MLP',MLPClassifier(hidden_layer_sizes=(15,15))))             #Logistic Regression
models.append(('LDA',LinearDiscriminantAnalysis()))    #Linear Discrimant Analysis
models.append(('KNN5|IBK',KNeighborsClassifier(n_neighbors=5)))          #K-NearestNeighbor Classifier
models.append(('KNN10|IBK',KNeighborsClassifier(n_neighbors=10)))          #K-NearestNeighbor Classifier
models.append(('KNN15|IBK',KNeighborsClassifier(n_neighbors=15)))          #K-NearestNeighbor Classifier
models.append(('CART|J48',DecisionTreeClassifier()))       #Decision Tree Classifier 
models.append(('RF',RandomForestClassifier(n_estimators=100)))       #Random Forest Classifier 
models.append(('NB',GaussianNB()))                     #Naive Bayes
models.append(('SVM',SVC()))                           #Support Vector Machine
results = []
names = []
for name,model in models:
    kfold = model_selection.KFold(n_splits=10,shuffle= False)
    cv_results = model_selection.cross_validate(model,train ,target,cv=kfold, scoring= scoring)
    results.append(cv_results)
    names.append(name)
    precision = '%s precision: %f (%f)' % (name, cv_results['test_precision'].mean()*100, cv_results['test_precision'].std())
    recall = '%s recall: %f (%f)' % (name, cv_results['test_recall'].mean()*100, cv_results['test_recall'].std())
    f1_score = '%s f1_score: %f (%f)' % (name, cv_results['test_f1_score'].mean()*100, cv_results['test_f1_score'].std())
    accuracy = '%s f1_score: %f (%f)' % (name, cv_results['test_accuracy'].mean()*100, cv_results['test_accuracy'].std())
    print("precision : ",precision)
    print("recall : ",recall)
    print("f1_score : ",f1_score)
    print("accuracy : ", accuracy)
# dict_keys(['fit_time', 'score_time', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1_score'])
# cross_validate or cross_val_score

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


precision :  LR precision: 89.707543 (0.015503)
recall :  LR recall: 86.024100 (0.012269)
f1_score :  LR f1_score: 87.811211 (0.007158)
accuracy :  LR f1_score: 87.252568 (0.007828)
precision :  MLP precision: 88.583116 (0.035310)
recall :  MLP recall: 90.367006 (0.054319)
f1_score :  MLP f1_score: 89.245261 (0.014427)
accuracy :  MLP f1_score: 88.440240 (0.010604)
precision :  LDA precision: 83.996167 (0.005925)
recall :  LDA recall: 74.520172 (0.012748)
f1_score :  LDA f1_score: 78.970661 (0.008793)
accuracy :  LDA f1_score: 78.825456 (0.007175)
precision :  KNN5|IBK precision: 90.765659 (0.006150)
recall :  KNN5|IBK recall: 89.603863 (0.009773)
f1_score :  KNN5|IBK f1_score: 90.179987 (0.007593)
accuracy :  KNN5|IBK f1_score: 89.589983 (0.007409)
precision :  KNN10|IBK precision: 91.742455 (0.006144)
recall :  KNN10|IBK recall: 87.862459 (0.008911)
f1_score :  KNN10|IBK f1_score: 89.758977 (0.006746)
accuracy :  KNN10|IBK f1_score: 89.304258 (0.006304)
precision :  KNN15|IBK precisi

In [ ]:
# # With Out Rewteet faeture
# models = []
# models.append(('LR',LogisticRegression()))             #Logistic Regression
# models.append(('LDA',LinearDiscriminantAnalysis()))    #Linear Discrimant Analysis
# models.append(('KNN|IBK',KNeighborsClassifier(n_neighbors=5)))          #K-NearestNeighbor Classifier
# models.append(('CART|J48',DecisionTreeClassifier()))       #Decision Tree Classifier 
# models.append(('RF',RandomForestClassifier(n_estimators=100)))       #Random Forest Classifier 
# models.append(('NB',GaussianNB()))                     #Naive Bayes
# models.append(('SVM',SVC()))                           #Support Vector Machine
# results = []
# names = []
# for name,model in models:
#     kfold = model_selection.KFold(n_splits=10,random_state=seed,shuffle= True)
#     cv_results = model_selection.cross_validate(model,train_wtRT ,target_wtRT,cv=kfold, scoring= scoring)
#     results.append(cv_results)
#     names.append(name)
#     precision = '%s precision: %f (%f)' % (name, cv_results['test_precision'].mean()*100, cv_results['test_precision'].std())
#     recall = '%s recall: %f (%f)' % (name, cv_results['test_recall'].mean()*100, cv_results['test_recall'].std())
#     f1_score = '%s f1_score: %f (%f)' % (name, cv_results['test_f1_score'].mean()*100, cv_results['test_f1_score'].std())
#     print(precision)
#     print(recall)
#     print(f1_score)
# # dict_keys(['fit_time', 'score_time', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1_score'])
# # cross_validate or cross_val_score

In [ ]:
#USING CROSS VALIDATION TO ESTIMATE THE BEST ALGOTRITHM 
models = []
models.append(('LR',LogisticRegression()))             #Logistic Regression
models.append(('MLP',MLPClassifier(hidden_layer_sizes=(15,15))))             #Logistic Regression
models.append(('LDA',LinearDiscriminantAnalysis()))    #Linear Discrimant Analysis
models.append(('KNN5',KNeighborsClassifier()))          #K-NearestNeighbor Classifier
models.append(('KNN10|IBK',KNeighborsClassifier(n_neighbors=10)))          #K-NearestNeighbor Classifier
models.append(('KNN15|IBK',KNeighborsClassifier(n_neighbors=15)))          #K-NearestNeighbor Classifier
models.append(('CART',DecisionTreeClassifier()))       #Decision Tree Classifier 
models.append(('RF',RandomForestClassifier(n_estimators=100)))       #Random Forest Classifier 
models.append(('NB',GaussianNB()))                     #Naive Bayes
models.append(('SVM',SVC()))                           #Support Vector Machine
results = []
names = []
for name,model in models:
    kfold = model_selection.KFold(n_splits=10,random_state=seed,shuffle= True)
    cv_results = model_selection.cross_validate(model,train ,target,cv=kfold, scoring= scoring)
    results.append(cv_results)
    names.append(name)
    precision = '%s precision: %f (%f)' % (name, cv_results['test_precision'].mean()*100, cv_results['test_precision'].std())
    recall = '%s recall: %f (%f)' % (name, cv_results['test_recall'].mean()*100, cv_results['test_recall'].std())
    f1_score = '%s f1_score: %f (%f)' % (name, cv_results['test_f1_score'].mean()*100, cv_results['test_f1_score'].std())
    accuracy = '%s f1_score: %f (%f)' % (name, cv_results['test_accuracy'].mean()*100, cv_results['test_accuracy'].std())
    print("precision : ",precision)
    print("recall : ",recall)
    print("f1_score : ",f1_score)
    print("accuracy : ", accuracy)
# dict_keys(['fit_time', 'score_time', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1_score'])
# cross_validate or cross_val_score

NameError: ignored